In [16]:
import pandas as pd
from utils.db import engine  # your SQLAlchemy engine

# 1) Write your ad‐hoc SQL
sql = """
WITH
  coach_cte AS (
    SELECT
      u.club_id,
      CASE
        WHEN EXTRACT(MONTH FROM u.created_at) >= 7 THEN
          CONCAT(EXTRACT(YEAR FROM u.created_at)::INT, '/', EXTRACT(YEAR FROM u.created_at)::INT + 1)
        ELSE
          CONCAT((EXTRACT(YEAR FROM u.created_at)::INT - 1), '/', EXTRACT(YEAR FROM u.created_at)::INT)
      END AS season,
      COUNT(*) AS coaches
    FROM users u
    WHERE u.role    = 'coach'
      AND u.active  IS TRUE
      AND u.club_id IS NOT NULL
    GROUP BY u.club_id, season
  ),

  parent_cte AS (
    SELECT
      s.club_id,
      CASE
        WHEN EXTRACT(MONTH FROM g.created_at) >= 7 THEN
          CONCAT(EXTRACT(YEAR FROM g.created_at)::INT, '/', EXTRACT(YEAR FROM g.created_at)::INT + 1)
        ELSE
          CONCAT((EXTRACT(YEAR FROM g.created_at)::INT - 1), '/', EXTRACT(YEAR FROM g.created_at)::INT)
      END AS season,
      COUNT(DISTINCT g.id) AS parents
    FROM users g
    JOIN users s
      ON g.id = CAST(s.guardian_id AS INTEGER)
    WHERE g.role    = 'guardian'
      AND g.active  IS TRUE
      AND s.club_id IS NOT NULL
    GROUP BY s.club_id, season
  ),

  skier_cte AS (
    SELECT
      u.club_id,
      CASE
        WHEN EXTRACT(MONTH FROM u.created_at) >= 7 THEN
          CONCAT(EXTRACT(YEAR FROM u.created_at)::INT, '/', EXTRACT(YEAR FROM u.created_at)::INT + 1)
        ELSE
          CONCAT((EXTRACT(YEAR FROM u.created_at)::INT - 1), '/', EXTRACT(YEAR FROM u.created_at)::INT)
      END AS season,
      COUNT(*) AS skiers
    FROM users u
    WHERE u.role    = 'skier'
      AND u.active  IS TRUE
      AND u.club_id IS NOT NULL
    GROUP BY u.club_id, season
  ),

  eval_cte AS (
    SELECT
      u.club_id,
      CASE
        WHEN EXTRACT(MONTH FROM cr.created_at) >= 7 THEN
          CONCAT(EXTRACT(YEAR FROM cr.created_at)::INT, '/', EXTRACT(YEAR FROM cr.created_at)::INT + 1)
        ELSE
          CONCAT((EXTRACT(YEAR FROM cr.created_at)::INT - 1), '/', EXTRACT(YEAR FROM cr.created_at)::INT)
      END AS season,
      COUNT(*) AS evaluations_completed
    FROM coach_rankings cr
    JOIN users u
      ON cr.coach_id = u.id
    WHERE u.club_id IS NOT NULL
    GROUP BY u.club_id, season
  ),

  drill_cte AS (
    SELECT
      u.club_id,
      CASE
        WHEN EXTRACT(MONTH FROM sd.created_at) >= 7 THEN
          CONCAT(EXTRACT(YEAR FROM sd.created_at)::INT, '/', EXTRACT(YEAR FROM sd.created_at)::INT + 1)
        ELSE
          CONCAT((EXTRACT(YEAR FROM sd.created_at)::INT - 1), '/', EXTRACT(YEAR FROM sd.created_at)::INT)
      END AS season,
      COUNT(*) AS drills_shared
    FROM share_drills sd
    JOIN users u
      ON sd.coach_id = u.id
    WHERE u.club_id IS NOT NULL
    GROUP BY u.club_id, season
  ),

  contact_cte AS (
    SELECT
      club_id,
      season,
      primary_contact,
      primary_contact_email
    FROM (
      SELECT
        u.club_id,
        (u.firstname || ' ' || u.lastname) AS primary_contact,
        u.email                          AS primary_contact_email,
        CASE
          WHEN EXTRACT(MONTH FROM u.created_at) >= 7 THEN
            CONCAT(EXTRACT(YEAR FROM u.created_at)::INT, '/', EXTRACT(YEAR FROM u.created_at)::INT + 1)
          ELSE
            CONCAT((EXTRACT(YEAR FROM u.created_at)::INT - 1), '/', EXTRACT(YEAR FROM u.created_at)::INT)
        END                               AS season,
        ROW_NUMBER() OVER (
          PARTITION BY u.club_id,
                       CASE
                         WHEN EXTRACT(MONTH FROM u.created_at) >= 7 THEN
                           CONCAT(EXTRACT(YEAR FROM u.created_at)::INT, '/', EXTRACT(YEAR FROM u.created_at)::INT + 1)
                         ELSE
                           CONCAT((EXTRACT(YEAR FROM u.created_at)::INT - 1), '/', EXTRACT(YEAR FROM u.created_at)::INT)
                       END
          ORDER BY u.created_at
        )                                  AS rn
      FROM users u
      WHERE u.role    = 'coach'
        AND u.active  IS TRUE
        AND u.club_id IS NOT NULL
    ) sub
    WHERE rn = 1
  ),

  seasons_union AS (
    SELECT club_id, season FROM coach_cte
    UNION
    SELECT club_id, season FROM parent_cte
    UNION
    SELECT club_id, season FROM skier_cte
  )

SELECT
  c.id                         AS club_id,
  su.season,
  c.name                       AS club_name,
  c.ptso                       AS ptso,
  COALESCE(co.coaches, 0)                       AS coaches,
  COALESCE(pa.parents, 0)                       AS parents,
  COALESCE(sk.skiers, 0)                        AS skiers,
  COALESCE(ev.evaluations_completed, 0)         AS evaluations_completed,
  COALESCE(dr.drills_shared, 0)                 AS drills_shared,
  COALESCE(ct.primary_contact, '')              AS primary_contact,
  COALESCE(ct.primary_contact_email, '')        AS primary_contact_email,
  'Active'                                     AS status
FROM clubs c
JOIN seasons_union su
  ON su.club_id = c.id
LEFT JOIN coach_cte co
  ON co.club_id = c.id AND co.season = su.season
LEFT JOIN parent_cte pa
  ON pa.club_id = c.id AND pa.season = su.season
LEFT JOIN skier_cte sk
  ON sk.club_id = c.id AND sk.season = su.season
LEFT JOIN eval_cte ev
  ON ev.club_id = c.id AND ev.season = su.season
LEFT JOIN drill_cte dr
  ON dr.club_id = c.id AND dr.season = su.season
LEFT JOIN contact_cte ct
  ON ct.club_id = c.id AND ct.season = su.season
ORDER BY su.season, c.name;



"""

# 2) Load into a DataFrame
df = pd.read_sql(sql, engine)

# 3) Peek at the raw data
df.head(100)


,club_id,season,club_name,ptso,coaches,parents,skiers,evaluations_completed,drills_shared,primary_contact,primary_contact_email,status
0,41,2024/2025,Adanac Ski Club,ontario,6,1,2,0,0,Bradley Laurin,blaurin08@icloud.com,Active
1,1,2024/2025,Alberta Alpine Ski Association,alberta,2,0,1,0,0,Curtis Rousseau,curtisrousseau@icloud.com,Active
2,2,2024/2025,Alberta Masters Ski Club,alberta,2,0,1,0,0,Andrew Ross,amhross@gmail.com,Active
3,134,2024/2025,Alberta Ski Cross,alberta,0,0,1,0,0,,,Active
4,133,2024/2025,Alpin Canada Ascension,canada,34,6,22,4,0,meribeth deen,meribeth.deen@gmail.com,Active
...,...,...,...,...,...,...,...,...,...,...,...,...
95,208,2024/2025,Equipe De Course Edelweiss,quebec,8,1,2,2,0,Mario Hagnon,mpgagnonski@gmail.com,Active
96,123,2024/2025,Équipe De Ski Camp Fortune,quebec,3,3,5,0,0,Francois couture,coutsf60@gmail.com,Active
97,124,2024/2025,Équipe De Ski Force G Inc.,quebec,7,19,35,7,0,Aurélie messier,headcoachgleason@gmail.com,Active
98,209,2024/2025,Équipe De Ski La Tuque,quebec,1,0,1,0,0,Felix Lavergne,felix.lavergne@hotmail.ca,Active


,club_id,season,primary_contact,primary_contact_email
0,1,2024/2025,Curtis Rousseau,curtisrousseau@icloud.com
1,2,2024/2025,Andrew Ross,amhross@gmail.com
2,3,2024/2025,Blais O'Brien,u14headcoach@banffalpineracers.com
3,4,2024/2025,Brent MacDonald,james.brent.macdonald@gmail.com
4,6,2024/2025,Jennifer Cooke,jennifer.cooke8@gmail.com
5,7,2024/2025,Suzanne Sorensen,mrsblizzard@shaw.ca
6,8,2024/2025,Jarred Pawlowski,jarred.pawlowski@gmail.com
7,9,2024/2025,Mark Stein,m.stein@shaw.ca
8,11,2024/2025,matthew vance,mattdvance@msn.com
9,13,2024/2025,Hannah Penkala,hannahpenkala@gmail.com


In [10]:
df

,level_name,skier_count
